# Lab 6: Particle Filter

## Implementation (25 Pts)

In this lab, you will implement a particle filter for performing state estimation. Consider a system whose state is given by $\bar{x} = [x,y]$, where $x$ is the x-position of our system and $y$ is the y-position. Assume that we have a perfect model of the dynamics of the system:
$$x_{t+1} = x_t + 0.1 x_t,$$
$$y_{t+1} = y_t - 0.2 y_t.$$
Suppose that we have a sensor that provides an (imperfect) estimate of the distance of the state from the origin. After thorough experimentation, we have found that the following is a good model of our sensor:
$$p(z_t | \bar{x}_t) = \mathcal{N}(\|\bar{x}_t\|, \sigma_\text{meas}^2),$$
where $\sigma_\text{meas}^2$ is equal to 0.2. In other words, the sensor measurement is a Gaussian random variable with mean equal to the distance $\|\bar{x}_t\|$ from the origin and the standard deviation equal to $\sigma_\text{meas}$. 

Suppose further that our initial belief about the state is defined by a Gaussian distribution with mean $[0, 0]$ and covariance equal to the $2 \times 2$ identity matrix, i.e. $\mathcal{N}(\bar{0}, I)$.

At each point in time, the true state $\bar{x}_t$ of the system evolves according to the dynamics given above. But, we do not have a perfect estimate of the state. Instead, we must use the sensor measurements $z_t$ we receive to estimate the state. We will use a particle filter to (approximately) maintain and update a belief over the state. Your task is to fill out the portions of the code below marked "TODO" to complete the particle filter implementation. First, we will  import libraries and setup the dynamics:  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import scipy.stats as st
from typing import List, Tuple
from IPython.display import HTML, Image

%matplotlib notebook

################ Setup dynamics and sensor (Do not modify) ###########

def update_state(state: np.ndarray) -> np.ndarray:
    """
    Update the state using the (known) dynamics of the system.
    
    @param state: The current state of the system.
    
    @return: The updated state of the system.
    """
    state[0] = state[0] + 0.1 * state[0]
    state[1] = state[1] - 0.2 * state[1]
    return state

def get_sensor_reading(state_true: np.ndarray, sensor_std_dev: float) -> float:
    """
    Obtain a sensor measurement. This function simulates the sensor that the robot has.
    
    @param state_true: The true state of the robot.
    @param sensor_std_dev: The standard deviation of the sensor's noise.
    
    @return: A sensor measurement
    """
    z = np.random.normal(np.linalg.norm(state_true), sensor_std_dev) 
    return z

Below, you will implement the particle filter. Note that your implementation may take a few seconds to run. This is because we are storing all the particle information for the whole simulation and doing so in an inefficient manner which is convenient for plotting later. In practice, particle filters run very efficiently.

In [ ]:
def particle_filter(initial_state: np.ndarray, num_particles: int, 
                    sensor_std_dev: float, init_mean: np.ndarray,
                    init_cov: np.ndarray, horizon: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Simulate a particle filter over a time horizon.
    
    @param initial_state: The robot's true initial state as a 2-vector.
    @param num_particles: The number of particles in the filter.
    @param sensor_std_dev: The standard deviation of the sensor model's output.
    @param init_mean: The mean of the initial distribution for the model as a 2-vector.
    @param init_cov: The covariance of the initial distribution as a 2-by-2 matrix.
    @param horizon: The number of steps to simulate the system for.
    
    @return: A 2-tuple whose first entry contains the history of the true state value of
             the system and whose second entry contains the history of particle values
             for the system.
    """
    weights = np.zeros(num_particles)
    state_true = initial_state.copy()
    
    particles = np.random.multivariate_normal(init_mean, init_cov, num_particles)

    particle_history = [particles.copy()]
    state_true_history = [state_true.copy()]

    for t in range(horizon): # for each time-step t = 0,1,2,...

        ######### Do not modify #############
        # Update true state
        state_true = update_state(state_true)
        state_true_history.append(state_true.copy())

        # Receive sensor measurement
        z_t = get_sensor_reading(state_true, sensor_std_dev)
        #####################################

        ####### Particle filter code ########
        for i in range(num_particles): # For each particle
            ##### TODO: Dynamics update (5 pts) #####            
            #####################################################

        ##### TODO: Implement resampling step (15 pts) #################    
        ###########################################################

        particle_history.append(particles.copy())
    
    return state_true_history, particle_history    

This cell actually runs the filter.

In [ ]:
# Number of particles 
num_particles = 1000

# Initialize particles using initial belief
init_mean = [0.0, 0.0]
init_cov = [[5.0, 0], [0, 5.0]]  # covariance is the identity matrix

# Initialize true state
state_true = np.array([1.0, 1.0])

# Define sensor noise standard deviation
sensor_std_dev = 0.2

# Time horizon
horizon = 15

state_true_history, particle_history = particle_filter(state_true, num_particles, sensor_std_dev, init_mean, init_cov, horizon)

Finally, this cell animates the results. If you get an error rendering the video, you may need to install FFMPEG. If the cell runs without error but the animation is not displayed, check the directory where the notebook is saved for the anim.gif file. Some browsers (including Microsoft Edge) may not properly display the animation despite the file being created and saved correctly.

In [ ]:
plt.ioff()
fig, ax = plt.subplots()

ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
ax.set_aspect('equal')

particle_scatter = ax.scatter([], [])
state_scatter = ax.scatter([], [])    

def animate(i):
    particle_scatter.set_offsets(particle_history[i])
    state_scatter.set_offsets(state_true_history[i])
    
# ani = animation.FuncAnimation(fig, animate, frames=len(particle_history))
import matplotlib
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(particle_history))

ani.save('./anim.gif', writer='pillow', fps=100)
Image(url='./anim.gif')

## Discussion (5 Pts)

If your particle filter is implemented correctly, you should notice that the particles cluster in two locations. In the cell below, briefly explain why this behavior occurs, and how you might change the system so that the filter converges to the correct state.

### Your Answer:

